In [272]:
import scipy.io
import numpy as np
import scipy.optimize
import datetime 
import ephem
import random
from scipy import spatial
import matplotlib.pyplot as plt
%matplotlib
from astropy.table import Table
import astropy.units as u
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages
import astropy
from astropy.time import Time
from astropy.coordinates import EarthLocation
import sunpy.coordinates
from datetime import datetime, timezone
from math import atan, atan2, degrees, floor, pi, radians, sin, sqrt
from skyfield.api import earth, JulianDate, now, sun
data=scipy.io.loadmat('BNS-sources-trial-10saleem.mat')['data'] 
#sourceparameters
DL=data[:,0]
theta=data[:,1]
phi=data[:,2]
print(len(DL),len(theta),len(phi))

Using matplotlib backend: MacOSX
220000 220000 220000


In [13]:
snrH=data[:,5]
snrL=data[:,6]
snrV=data[:,7]
snrK=data[:,8]
snrI=data[:,9]
#or for networks of 5 and 3 detectors, one can use directly,
snrLHVKI=np.sqrt(data[:,5]**2 + data[:,6]**2 + data[:,7]**2 +data[:,8]**2 + data[:,9]**2)  
# SNR at LHVKI
snrLHV=np.sqrt(data[:,5]**2 + data[:,6]**2 + data[:,7]**2 )  # LHV*
snrLHVK = np.sqrt(data[:,5]**2 + data[:,6]**2 + data[:,7]**2 + data[:,8]**2)
snrLHVI = np.sqrt(data[:,5]**2 + data[:,6]**2 + data[:,7]**2 + data[:,9]**2)
len(snrLHVKI),len(snrLHVK)

(220000, 220000)

In [14]:
from astropy.time import Time
time1 = Time('2022-01-01T00:00:00.123456789') # ask gaurav why
time2 = Time('2023-01-01T00:00:00.123456789')
#%time homotime = Time(np.linspace(time1.jd, time2.jd, len(theta)), format="jd")
%time homotime = Time(np.random.uniform(low=time1.jd, high=time2.jd, size=len(theta)), format="jd").sort()

CPU times: user 161 ms, sys: 28.6 ms, total: 189 ms
Wall time: 185 ms


In [310]:
length = 612

In [22]:
sunloc = astropy.coordinates.get_sun(homotime)  #getting loaction of sun with respect to earth

In [23]:
sunRA = sunloc.ra.deg  #sun's RA in earth's frame in degrees
sunDEC = sunloc.dec.deg #sun's DEC in earth's frame in degrees
sunDIST = sunloc.distance.AU*1.496e8 #sun distance from earth in km

In [67]:
(homotime[1].sidereal_time('mean','greenwich') - homotime[0].sidereal_time('mean','greenwich')).rad

0.0019036382527979978

In [285]:
plt.plot(homotime.value,sunRA)

In [447]:
plt.plot(homotime.value[0:598],sunRA[0:598])

In [452]:
plt.plot(homotime.value[0:598],f)

In [414]:
(homotime.sidereal_time('mean','greenwich'))[1:5].rad

array([1.76605593, 1.77769078, 1.78040752, 1.79898414])

In [294]:
def radectolatlong(Ra,Dec,time): #input in degrees
    ra = np.deg2rad(Ra)
    dec = np.deg2rad(Dec)
    time_sid = (time.sidereal_time('mean','greenwich')).rad
    lat = dec
    long = ra - time_sid 
    Lat = np.rad2deg(lat)
    Long = np.rad2deg(long)-100.46
    return Lat,Long #output in degrees

def latlongtoradec(Lat,Long,time): #input in degrees
    lat = np.deg2rad(Lat)
    long = np.deg2rad(Long)
    time_sid = (time.sidereal_time('mean','greenwich')).rad
    
    DEC = lat
    RA = long + time_sid
    ra = np.rad2deg(RA)+100.46       #The value of 100.46 degrees is the value needed to make the -
                                      #expression yield the correct value for GMST at 0h UT on 1 January 2000.
    dec = np.rad2deg(DEC)
    return ra,dec #output in degrees



def is_saa(lat, longi, time):
    """Return whether the satellite is in South Atlantic Anamoly or not.
    Hard bounds used from the data of RXTE and assumed that the SAA region is rectangular below the inclination of RXTE.
    
    ra - right ascention of the body in degrees
    dec - declination of the body in degrees
    time - UTC time in datetime format
    """
    # lat, longi = radec2latlong(ra, dec, time)
    #lat, longi = eci_ecef(ra, dec, time)
    lat = np.rad2deg(lat)
    longi = np.rad2deg(longi)
    if -23< lat <= -7 and -90<=longi<=-10:
        return True
    
#     if -90<=longi<=-10:
#         y = 7*longi/80 + 23/8
#         if lat < y:
#             return True
    if -100<=longi<=-35:
        y=14*longi/65+189/13
        if lat<=y:
            return True
    
    if -35<longi<-10:
        y=-14/25*longi-63/5
        if lat<y:
            return True
    
    
    if -10<=longi<=40:
        z = -9*longi/25 -43/5
        if lat<z: 
            return True
	

    if -50<= lat <=-23 and -90<=longi<=40:
        return True
    else: return False 

def getxyz(theta,phi,n):
    if n==1:
        #lat, long
        theta = np.pi/2 - theta
    x=np.sin(theta)*np.cos(phi)
    y=np.sin(theta)*np.sin(phi)
    z=np.cos(theta)
    return x,y,z
def getnorm(vec):
    norm = np.linalg.norm(vec)
    #print (norm)
    return vec/norm

In [295]:
timeinISO = str(homotime.iso)

In [296]:
line0 = 'ASTROSAT' #Daksha
line1 = '1 40930U 15052A   22302.17850942  .00000772  00000-0  19439-4 0  9990'
line2 = '2 40930   5.9975   0.5159 0009292 339.1937  20.7998 14.76165238112787'
line3 = '2 40930   5.9975   0.5159 0009292 339.1937 200.7998 14.76165238112787'

# line0 = 'SWIFT'#BAT
# line1 = '1 28485U 04047A   22302.18405080  .00001301  00000-0  60140-4 0  9997'
# line2 = '2 28485  20.5572 277.8742 0011565 130.4507 229.6936 15.03911323708409'
# line0 = 'FGRST (GLAST)'#Fermi
# line1 = '1 33053U 08029A   22100.45970249  .00000511  00000-0  12538-4 0  9998'
# line2 = '2 33053  25.5819 140.4060 0012690 120.1881 239.9912 15.11069703542313'
astrosat1 = ephem.readtle(line0,line1,line2)
astrosat2 = ephem.readtle(line0,line1,line3)

sublat1 = np.empty(len(theta),dtype = object)
sublong1 = np.empty(len(theta),dtype = object)
sublat2 = np.empty(len(theta),dtype = object)
sublong2 = np.empty(len(theta),dtype = object)
for i in range(len(theta)):
    time = str(homotime[i].iso)
    astrosat1.compute(time)
    astrosat2.compute(time)
    sublat1[i] = (astrosat1.sublat)#sublat in radians
    sublong1[i] = (astrosat1.sublong)#sublong in radian
    sublat2[i] = (astrosat2.sublat)#sublat in radians
    sublong2[i] = (astrosat2.sublong)#sublong in radian

In [297]:
time = datetime(2015, 1, 1, tzinfo=timezone.utc)
x, y, z = earth(JulianDate(utc=time)).observe(sun).apparent().position.AU

In [ ]:
for i in range(10):
    astrosat1.compute(str(homotime[i].iso))
    (np.array(astrosat1.sublat))

In [298]:
RA1 = np.empty(len(theta),dtype = object)
DEC1 = np.empty(len(theta),dtype = object)
for i in range(len(theta)):
    time = str(homotime[i].iso)
    astrosat1.compute(time)
    RA1[i] = np.rad2deg(astrosat1.ra)
    DEC1[i] = (astrosat1.dec)

In [319]:
plt.plot(homotime.value[0:length],RA1[0:length],color="green")
plt.xlabel("1 day",color="blue")
plt.ylabel("Right ascension in degree (in degree)",color="blue")
plt.title("DAKSHA",color="blue")
plt.savefig("RA_satellite.pdf")

In [312]:
plt.plot(homotime.value[0:length],DEC1[0:length],"green")
plt.xlabel("1 day",color="blue")
plt.ylabel("Declination or Latitude (in degree)",color="blue")
plt.title("DAKSHA",color="blue")
plt.savefig("Declination_satellite.pdf")

#### HA0 = LST0 - RA0
#### HA1 = HA0 = LST1 - RA1
#### RA1 = RA0 + (LST1 - LST0)



In [198]:
(radectolatlong(sunRA[i],sunDEC[i],homotime[i])[1])

284.46812108191256

In [317]:
earthRA = np.empty((length),dtype=object)
earthDEC = np.empty(length,dtype=object)
for i in range((length)):
    earthRA[i] = latlongtoradec(sublat1[i],sublong1[i],homotime[i])[0]
    earthDEC[i] = latlongtoradec(sublat1[i],sublong1[i],homotime[i])[1]

In [318]:
plt.plot(homotime.value[0:length],earthRA,"green")
plt.xlabel("1 day",color="blue")
plt.ylabel("Right ascension in degree",color="blue")
plt.title("RA for anti-earth orbit",color="blue")
plt.savefig("RA_satellite.pdf")

## SUN

In [95]:
time = datetime(2022, 1, 1)
x, y, z = earth(JulianDate(utc=time)).observe(sun).apparent().position.AU

In [320]:
x = np.empty(length,dtype=object)
y = np.empty(length,dtype=object)
z = np.empty(length,dtype=object)
satvecold = np.empty(length,dtype=object)
xaxis = np.empty(length,dtype=object)
for i in range(0,length-1):
    temp = homotime.iso[i]
    year = int(temp[0:4])
    month = int(temp[5:7])
    day = int(temp[8:10])
    hour = int(temp[11:13])
    minu = int(temp[14:16])
    secs = int(temp[17:19])
    #time = Time(time)
    time = datetime(year, month,day,hour,minu,secs)
    #time0 = datetime.datetime(2000, 1, 1, 12, 0, 0)
    #time = datetime(2022, 1, 1, tzinfo=timezone.utc)
    x[i]=(-1*1.496e8*earth(JulianDate(utc=time)).observe(sun).apparent().position.AU[0])
    y[i]=(-1*1.496e8*earth(JulianDate(utc=time)).observe(sun).apparent().position.AU[1])
    z[i]=(-1*1.496e8*earth(JulianDate(utc=time)).observe(sun).apparent().position.AU[2])
    
    temp = homotime.iso[i+1]
    year = int(temp[0:4])
    month = int(temp[5:7])
    day = int(temp[8:10])
    hour = int(temp[11:13])
    minu = int(temp[14:16])
    secs = int(temp[17:19])
    #time = Time(time)
    time = datetime(year, month,day,hour,minu,secs)
    x1 = (-1*1.496e8*earth(JulianDate(utc=time)).observe(sun).apparent().position.AU[0])
    y1 = (-1*1.496e8*earth(JulianDate(utc=time)).observe(sun).apparent().position.AU[1])
    z1 = (-1*1.496e8*earth(JulianDate(utc=time)).observe(sun).apparent().position.AU[2])
    xaxis[i] = x1-x[i],y1-y[i],z1-z[i]
    time = str(homotime[i].iso)
    astrosat1.compute(time)
    s = (getnorm(getxyz(astrosat1.sublat,astrosat1.sublong,1)))
    satvecold[i] = np.array([s[0]*7000,s[1]*7000,s[2]*7000])

In [333]:
homotime.iso[500]

'2022-01-01 19:22:52.730'

In [321]:
sunsat = np.empty(length,dtype=object)
for i in range(0,length-1):
    sunsat[i] = x[i]+satvecold[i][0],y[i]+satvecold[i][1],z[i]+satvecold[i][2]

In [322]:
decsun = np.empty(length,dtype=object)
for i in range(0,length-1):
    norm1 = (x[i]**2+y[i]**2+z[i]**2)**0.5
    norm2 = (sunsat[i][0]**2+sunsat[i][1]**2+sunsat[i][2]**2)**0.5
    decsun[i] = np.arccos((x[i]*sunsat[i][0]+y[i]*sunsat[i][1]+z[i]*sunsat[i][2])/(norm1*norm2))*180/np.pi

In [338]:
plt.plot(f[0:length-1],decsun[0:length-1])
#plt.xticks([0,50000,110000,160000,219999],['Jan','Mar','Jun','Sep','Dec'])
plt.xticks([0,100,200,300,400,500,610],['00:00','04:11','07:35','11:34','15:00','19:22','23:52'])
plt.title("Daksha's DEC w.r.t sun")
plt.xlabel("1 Day(time)")
plt.ylabel("DEC in degrees")
plt.savefig("DEC_Dakshawrtsunforoneday.pdf")

In [264]:
projsunsatonxy = np.empty(length,dtype=object)
rightascension = np.empty(length,dtype=object)
for i in range(0,length-1):
    sindel = np.sin(decsun[i]*np.pi/180)
    projsunsatonxy[i] = sindel*sunsat[i][0],sindel*sunsat[i][1],sindel*sunsat[i][2]
    norm1 = (projsunsatonxy[i][0]**2+projsunsatonxy[i][1]**2+projsunsatonxy[i][2])**0.5
    norm2 = (xaxis[0][0]**2+xaxis[0][1]**2+xaxis[0][2]**2)**0.5
    rightascension[i] = np.arccos((projsunsatonxy[i][0]*xaxis[0][0]+projsunsatonxy[i][1]*xaxis[0][1]+projsunsatonxy[i][2]*xaxis[0][2])/(norm1*norm2))*180/np.pi

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in arccos
  


In [340]:
plt.plot(f[0:length-1],rightascension[0:length-1])
plt.title("Daksha's RA w.r.t sun")
plt.xticks([0,100,200,300,400,500,610],['00:00','04:11','07:35','11:34','15:00','19:22','23:52'])
plt.xlabel("1 Day(time)")
plt.ylabel("RA in degrees")
plt.savefig("RA_Dakshawrtsunforoneday.pdf")

In [247]:
norm1 = (projsunsatonxy[60000][0]**2+projsunsatonxy[60000][1]**2+projsunsatonxy[60000][2])**0.5
norm2 = (xaxis[0][0]**2+xaxis[0][1]**2+xaxis[0][2]**2)**0.5
(projsunsatonxy[60000][0]*xaxis[0][0]+projsunsatonxy[60000][1]*xaxis[0][1]+projsunsatonxy[60000][2]*xaxis[0][2])/(norm1*norm2)

0.9947420214525368

In [255]:
f = np.linspace(0,220000,220000)

In [270]:
def function(theta,x):
    return x-np.sin(theta)  #theta in radians

In [276]:
scipy.optimize.fsolve(function,0,args=1/2)

array([0.52359878])

In [288]:
angle = np.empty(length,dtype=object)
guess = 0
for i in range(length-1):
    norm1 = (projsunsatonxy[i][0]**2+projsunsatonxy[i][1]**2+projsunsatonxy[i][2])**0.5
    norm2 = (xaxis[0][0]**2+xaxis[0][1]**2+xaxis[0][2]**2)**0.5
    x = (projsunsatonxy[i][0]*xaxis[0][0]+projsunsatonxy[i][1]*xaxis[0][1]+projsunsatonxy[i][2]*xaxis[0][2])/(norm1*norm2)
    if x>1:
        x=1
    angle[i] = scipy.optimize.fsolve(function,guess,args=x)[0]*180/np.pi
    guess += np.pi/110000

/anaconda3/lib/python3.6/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/anaconda3/lib/python3.6/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


In [291]:
plt.plot(f[0:length-1],angle[0:length-1])
plt.title("Daksha's RA w.r.t sun")
plt.xlabel("1 year")
plt.ylabel("RA in degrees")
plt.savefig("RA_dakshawrtsun.pdf")